# Týden 8. Strojové učení - pokračování.

## Naivní Bayes

Klasifikátor Naivní Bayes je založen na Bayesově teorému a předpokládá podmíněnou nezávislost mezi rysy danými označením třídy. Je široce používán pro klasifikační úlohy, jako je filtrování spamu, analýza sentimentu a další.

Bayesova věta je formulována jako:

$$
P(A|B) = \frac{P(B|A) \times P(A)}{P(B)}
$$

Pro klasifikační problém s rysy $ \mathbf{x} = (x_1, x_2, \ldots, x_n) $ a značkou třídy $ y $ je cílem najít:

$$
P(y|\mathbf{x}) = \frac{P(\mathbf{x}|y) \times P(y)}{P(\mathbf{x})}
$$

V Naivním Bayesovi vycházíme z naivního předpokladu, že všechny rysy jsou podmíněně nezávislé vzhledem k $ y $, tedy:

$$
P(\mathbf{x}|y) = P(x_1|y) \times P(x_2|y) \times \ldots \times P(x_n|y)
$$

Předpovědí je pak třída $ y $, která maximalizuje $ P(y|\mathbf{x}) $.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.naive_bayes import GaussianNB

V tomto kódu se make_blobs používá k vytvoření syntetické datové sady se dvěma shluky. Na těchto datech je natrénován Gaussův model Naive Bayes (GaussianNB). K vizualizaci rozhodovací hranice je vytvořena mřížka meshgrid, která je zobrazena pomocí contourf.

Barevné oblasti představují předpovězenou třídu pro každou oblast prostoru příznaků a rozptýlené body jsou původní datové body, zbarvené podle jejich skutečné značky třídy.

In [ ]:
# Create synthetic data
X, y = make_blobs(n_samples=300, centers=2, random_state=42, cluster_std=2.0)

# Train Gaussian Naive Bayes model
gnb = GaussianNB()
gnb.fit(X, y)

# Create mesh grid for visualization
xx, yy = np.meshgrid(np.linspace(X[:, 0].min() - 1, X[:, 0].max() + 1, 100),
                     np.linspace(X[:, 1].min() - 1, X[:, 1].max() + 1, 100))

# Predict classes for each grid point
Z = gnb.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# Plot the decision boundary
plt.contourf(xx, yy, Z, alpha=0.8)
plt.scatter(X[:, 0], X[:, 1], c=y, edgecolors='k', marker='o', linewidth=1)
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Gaussian Naive Bayes Decision Boundary')
plt.show()


## Rozhodovací strom

Rozhodovací strom je stromová struktura podobná diagramu, kde vnitřní uzel představuje funkci, větev představuje rozhodovací pravidlo a každý listový uzel představuje výsledek. Jedná se o algoritmus učení pod dohledem, který se používá pro klasifikační i regresní úlohy.

Základní algoritmus pro sestavení rozhodovacího stromu je založen na rekurzivním rozdělení množiny dat na podmnožiny. K rozhodnutí o rozdělení lze použít různá kritéria, například informační zisk, Giniho nečistotu nebo snížení rozptylu. 

Informační zisk pro binární klasifikaci lze například vyjádřit jako:

$$
\text{Informační zisk} = \text{Entropie(rodič)} - \sum \left( \frac{n}{N} \times \text{Entropie(podřízený)} \right)
$$

Kde \( \text{Entropie}(S) \) pro množinu \( S \) obsahující \( p \) kladných případů a \( n \) záporných případů je definováno jako:

$$
\text{Entropie}(S) = -p\log_2(p) - n\log_2(n)
$$

### Kroky algoritmu
1. Vyberte atribut ze souboru dat.
2. Vypočítejte významnost atributu při rozdělení dat.
3. Rozdělte data na základě vybraného atributu.
4. Opakujte kroky 1-3 pro každou větev, dokud není splněna jedna z podmínek zastavení, například maximální hloubka, minimum vzorků na listu nebo práh nejlepšího rozdělení.


Níže je uveden kód k demonstraci rozhodovacího stromu na syntetické sadě dat s vizualizací rozhodovací hranice:

In [ ]:
from sklearn.datasets import make_moons
from sklearn.tree import DecisionTreeClassifier

# Create synthetic data
X, y = make_moons(n_samples=100, noise=0.2, random_state=42)

# Initialize Decision Tree and fit data
clf = DecisionTreeClassifier(max_depth=2, random_state=42)
clf.fit(X, y)

# Create mesh grid for visualization
xx, yy = np.meshgrid(np.linspace(X[:, 0].min() - 1, X[:, 0].max() + 1, 100),
                     np.linspace(X[:, 1].min() - 1, X[:, 1].max() + 1, 100))

# Predict classes for each grid point
Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# Plot decision boundary
plt.contourf(xx, yy, Z, alpha=0.8)
plt.scatter(X[:, 0], X[:, 1], c=y, edgecolors='k', marker='o', linewidth=1)
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Decision Tree Classifier')
plt.show()

V tomto příkladu je pomocí funkce `make_moons` vytvořena syntetická datová sada a rozhodovací strom (`DecisionTreeClassifier`) je natrénován s maximální hloubkou 2. Rozhodovací hranice je spolu s původními datovými body vizualizována. Každá barevná oblast v grafu představuje předpovězenou třídu pro danou oblast prostoru příznaků.

## Náhodný les
Náhodný les je metoda skupinového učení, která funguje tak, že vytváří množství rozhodovacích stromů. Čím více stromů je v "lese", tím je model robustnější. Náhodný les lze použít pro klasifikační i regresní úlohy.

Náhodný les se oproti jednomu rozhodovacímu stromu zlepšuje zprůměrováním nebo většinovým hlasováním jednotlivých stromů. Matematicky je pro klasifikaci konečná předpověď $ \hat{y} $ dána vztahem:

$$
\hat{y} = \text{mode}(\hat{y}_1, \hat{y}_2, \ldots, \hat{y}_n)
$$

kde $ n $ je počet stromů a $ \hat{y}_i $ je předpověď ze stromu $ i^{th} $. U regrese je to průměr výstupů jednotlivých stromů.

### Kroky algoritmu

1. Náhodně vyberte "k" prvků z celkem "m" prvků, kde $ k < m $.
2. Z "k" rysů vypočítejte uzel pomocí nejlepšího bodu rozdělení.
3. Rozdělte uzel na podřízené uzly pomocí nejlepšího rozdělení.
4. Opakujte kroky 1-3, dokud listové uzly neobsahují méně než předem stanovený počet vzorků nebo nedosáhnou přijatelné čistoty.
5. Sestavte "n" stromů pomocí kroků 1-4 a seskupte je tak, aby vytvořily les.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialize Random Forest Classifier and fit data
clf = RandomForestClassifier(n_estimators=20, random_state=42)
clf.fit(X, y)

# Create mesh grid for visualization
xx, yy = np.meshgrid(np.linspace(X[:, 0].min() - 1, X[:, 0].max() + 1, 100),
                     np.linspace(X[:, 1].min() - 1, X[:, 1].max() + 1, 100))

# Predict classes for each grid point
Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# Plot decision boundary
plt.contourf(xx, yy, Z, alpha=0.8)
plt.scatter(X[:, 0], X[:, 1], c=y, edgecolors='k', marker='o', linewidth=1)
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Random Forest Classifier')
plt.show()

V tomto příkladu je na syntetických datech natrénován RandomForestClassifier s 20 stromy (n_estimators=20). Hranice rozhodování je pak vizualizována stejným způsobem jako u jednoduchého rozhodovacího stromu.

Na rozdíl od jednoduchého rozhodovacího stromu využívá Random Forest ke konečné klasifikaci většinový hlas ze všech jednotlivých stromů. Jak je vidět, vede to k pružnější a robustnější rozhodovací hranici.

## Strojové učení bez dohledu

Strojové učení bez dohledu je typ strojového učení, při kterém se algoritmus učí z neoznačených dat. Na rozdíl od učení pod dohledem, kdy je každému trénovacímu příkladu přiřazen štítek, v případě učení bez dohledu pracuje model sám na sobě, aby odhalil přirozenou strukturu dat. Hlavním cílem je často modelovat rozložení dat tak, aby bylo možné tento model použít pro úlohy, jako třeba:

Shlukování: Algoritmy jako K-means, hierarchické shlukování a DBSCAN vytvářejí skupiny nebo "shluky" na základě přirozeného rozložení dat.

Snížení dimenzionality: Techniky, jako je analýza hlavních komponent (PCA) nebo autoenkodéry, snižují počet uvažovaných náhodných proměnných, přičemž zachovávají podstatné vlastnosti.

Asociace: Algoritmy jako Apriori a FP-growth se používají k nalezení pravidel, která popisují vztahy mezi zdánlivě nezávislými položkami v daném souboru dat.

Detekce anomálií: Algoritmy jako Isolation Forest a One-Class SVM se používají k detekci abnormálních vzorů, které neodpovídají očekávanému chování.

## K-means clustering

K-means clustering je neřízený algoritmus strojového učení, který se používá k rozdělení souboru dat do $ K $ shluků. Každý datový bod patří do shluku s nejbližší střední hodnotou.

Cílem K-means je minimalizovat součet čtverců uvnitř shluku (WCSS), který je definován jako:

$$
J = \sum_{i=1}^{K} \sum_{x \v C_i} || x - \mu_i ||^2
$$

kde $ \mu_i $ je centroid shluku $ C_i $, $ K $ je počet shluků a $ x $ je datový bod ve shluku $ C_i $.

### Kroky algoritmu
1. Zvolte $ K $ počátečních centroidů, jeden pro každý shluk. To lze provést náhodně nebo na základě heuristiky.
2. Přiřaďte každý datový bod k nejbližšímu centroidu a stane se členem tohoto shluku.
3. Vypočítejte nové centroidy jako střední hodnotu všech datových bodů ve shluku.
4. Opakujte kroky přiřazení a aktualizace, dokud nedojde k žádnému zlepšení, tj. centroidy se výrazně nezmění.

In [ ]:
from sklearn.cluster import KMeans

# Create synthetic data
X, _ = make_blobs(n_samples=300, centers=3, random_state=42, cluster_std=1.0)

# Initialize and fit KMeans algorithm
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(X)

# Fetch centroids and labels
centroids = kmeans.cluster_centers_
labels = kmeans.labels_

# Create meshgrid for visualization
xx, yy = np.meshgrid(np.linspace(X[:, 0].min() - 1, X[:, 0].max() + 1, 100),
                     np.linspace(X[:, 1].min() - 1, X[:, 1].max() + 1, 100))

# Predict cluster for meshgrid points
Z = kmeans.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# Plot
plt.contourf(xx, yy, Z, alpha=0.8)
plt.scatter(X[:, 0], X[:, 1], c=labels, edgecolors='k', marker='o', linewidth=1)
plt.scatter(centroids[:, 0], centroids[:, 1], c='red', marker='x')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('K-means Clustering')
plt.show()

## Analýza hlavních komponent (PCA) 

Analýza hlavních komponent (PCA) je technika redukce dimenzionality, která se používá k transformaci vysokodimenzionálních dat do méně dimenzionální podoby, přičemž se zachovává co největší část rozptylu původních dat.

### Základní matematika

Cílem PCA je najít novou sadu os, tzv. hlavních komponent, které maximalizují rozptyl v datech. Matematicky jde o rozklad vlastních čísel kovarianční matice dat $ \Sigma $:

$$
\Sigma V = V D
$$

Zde $ D $ je diagonální matice vlastních čísel $ \lambda $ a $ V $ obsahuje odpovídající vlastní vektory $ v $.

Hlavní komponenty jsou vlastní vektory seřazené sestupně podle vlastních čísel. První hlavní složka je vlastní vektor spojený s největší vlastní hodnotou a tak dále.

Data pak lze do tohoto nového prostoru promítnout pomocí:

$$
X_{\text{PCA}} = X V_k
$$

kde $ V_k $ obsahuje první $ k $ vlastní vektory a $ k $ je počet dimenzí, na které chcete data redukovat.

### Kroky algoritmu

1. **Centrování dat**: Odečtěte střední hodnotu každého prvku ze souboru dat.
2. **Vypočítejte kovarianční matici**: \( \Sigma = \frac{1}{N} X^T X \)
3. **Rozklad vlastních čísel**: Vyřešte \( V \) a \( D \).
4. **Třídění vlastních vektorů**: Seřaďte vlastní vektory podle klesajících vlastních hodnot.
5. **Projekce**: Vyberte první \( k \) vlastní vektory a promítněte data do nového prostoru.

In [ ]:
from sklearn.decomposition import PCA

# Create synthetic data
X, y = make_blobs(n_samples=300, centers=3, n_features=2, random_state=42)

# Fit PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

# Plot original data
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.scatter(X[:, 0], X[:, 1], c=y)
plt.title("Original Data")
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")

# Plot PCA-transformed data
plt.subplot(1, 2, 2)
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y)
plt.title("PCA Transformed Data")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")

plt.show()

V tomto příkladu máme syntetická data ve 2D prostoru, reprezentovaná symbolem `X`. Tato data transformujeme pomocí PCA do nového souřadnicového systému definovaného hlavními komponentami. Protože data jsou již ve 2D, budou transformovaná data také ve 2D.
Levý graf ukazuje původní datové body podbarvené jejich skutečnými značkami a pravý graf ukazuje datové body v novém souřadnicovém systému definovaném pomocí PCA.
Při pohledu na pravý graf je snadno vidět, že hlavní komponenty (osy) jsou orientovány ve směrech, kde se data nejvíce liší.
To by mělo poskytnout solidní pochopení PCA, matematiky, která za ní stojí, a její vizualizace.